# Central-Primitive Orthogonal Idempotents of $\mathbb{C}S_k$ as Actions on $(\mathbb{C}^n)^{\otimes k}$

We took linear algebra methods as far as they could go, so this time we will calculate the actions of the primitive-central, orthogonal idempotents of $\mathbb{C}S_k$ as they project onto the subspaces of $V=\mathbb{C^n}^{\otimes k}$ isomorphic to the simple $\mathfrak{gl}_n(\mathbb{C})\bigotimes \mathbb{C}S_k$ bimodules of $V$.

Luckily for us, SAGE already knows what these idempotents are. Let's begin by working with Symmetric Group Algebra. If we assign the group call *.algebra(F)

In [6]:
k=3
CSk=SymmetricGroup(k).algebra(QQbar)

In [7]:
CSk_idem=CSk.central_orthogonal_idempotents()
print CSk_idem

[1/6*() + 1/6*(2,3) + 1/6*(1,2) + 1/6*(1,2,3) + 1/6*(1,3,2) + 1/6*(1,3), 2/3*() - 1/3*(1,2,3) - 1/3*(1,3,2), 1/6*() - 1/6*(2,3) - 1/6*(1,2) + 1/6*(1,2,3) + 1/6*(1,3,2) - 1/6*(1,3)]


To multiply two elements together, you can simply write out $a*b$. To add them, write $a+b$. We interpret $()$ as the identity permutation, and can pass a permutation (python list) $[x_1,...,x_n]$ through to the algebra and return an element in the algebra using $CSk([x_1,...,x_n])$:

In [8]:
print CSk([2,1])
print CSk_idem[0]*CSk([2,1])

(1,2)
1/6*() + 1/6*(2,3) + 1/6*(1,2) + 1/6*(1,2,3) + 1/6*(1,3,2) + 1/6*(1,3)


With so much of the work already done for us, we just need to find a way to interpret an element in the algebra and build a true algebra homomorphism $\rho:\mathbb{C}S_k\rightarrow \text{End}(V)$.

In [9]:
CSk_basis=[p for p in SymmetricGroup(k)]
print CSk_basis

[(), (1,3,2), (1,2,3), (2,3), (1,3), (1,2)]


After building a basis set, we can obtain the $k!$- vector of coefficients from any idempotent.

In [10]:
s=CSk_idem[1]
v=zero_vector(QQbar, factorial(k))
print s
for i in range(0,factorial(k)):
    v[i]=s.coefficient(CSk_basis[i])
print v

2/3*() - 1/3*(1,2,3) - 1/3*(1,3,2)
(2/3, -1/3, -1/3, 0, 0, 0)


Now as long as we can define $\rho$ on each element of $S_k$, we can express the endomorphism as the sum of matrices with these coefficients. So let's bring back our $\rho$ function from last time:

In [11]:
def rho_local(g,n,k):
    z=zero_matrix(n^k)
    for j in range(0,n^k):
        l=NN(j).digits(n) 
        s=[0]
        while len((n^k-1).digits(n))-len(l) !=0:
            l=l+s 
        l.reverse() 
        perm="("+str(j)+","
        m=g(l)
        m.reverse() 
        y=0
        for i in range(len(l)):
            y+=m[i]*n^i
        perm+=str(y)+")"        
        for i in range(0,n^k):
            if i==y:
                z[i,j]=1
    return z

In [12]:
def vectorize_element(s,k):
    v=zero_vector(QQbar, factorial(k))
    for i in range(0,factorial(k)):
        v[i]=s.coefficient(CSk_basis[i])
    return v

Now, we can define a morphism on the symmetric group algebra:

In [13]:
def rho(s,n,k):
    vec=vectorize_element(s,k)
    mat=zero_matrix(QQbar,n^k)
    for i in range(0,factorial(k)):
        m=rho_local(CSk_basis[i],n,k)
        mat+=vec[i]*m
    return mat

Now we will use everything we made to see what our idempotent endomorphisms look like:

In [14]:
k=2
n=3
CSk=SymmetricGroup(k).algebra(QQbar)
CSk_idem=CSk.central_orthogonal_idempotents()
CSk_basis=[p for p in SymmetricGroup(k)]

for s in CSk_idem:
    M= rho(s,n,k)
    print M
    print "The rank of this matrix is: " + str(M.rank())


[  1   0   0   0   0   0   0   0   0]
[  0 1/2   0 1/2   0   0   0   0   0]
[  0   0 1/2   0   0   0 1/2   0   0]
[  0 1/2   0 1/2   0   0   0   0   0]
[  0   0   0   0   1   0   0   0   0]
[  0   0   0   0   0 1/2   0 1/2   0]
[  0   0 1/2   0   0   0 1/2   0   0]
[  0   0   0   0   0 1/2   0 1/2   0]
[  0   0   0   0   0   0   0   0   1]
The rank of this matrix is: 6
[   0    0    0    0    0    0    0    0    0]
[   0  1/2    0 -1/2    0    0    0    0    0]
[   0    0  1/2    0    0    0 -1/2    0    0]
[   0 -1/2    0  1/2    0    0    0    0    0]
[   0    0    0    0    0    0    0    0    0]
[   0    0    0    0    0  1/2    0 -1/2    0]
[   0    0 -1/2    0    0    0  1/2    0    0]
[   0    0    0    0    0 -1/2    0  1/2    0]
[   0    0    0    0    0    0    0    0    0]
The rank of this matrix is: 3


Just as we had hoped, the rank sums up to the dimension of $V$. We will now try various values of n, k and see if the column spaces of these matrices intersect trivially, thereby telling us that $V$ is the direct sum of projected spaces, and check if each projected space is invariant under the generators of $S_k$, telling us that $V$ has a subspace that decomposes as an $\mathbb{C}S_k$ module. Lastly, we see if the ranks of the matrices sum up to the dimension of $V$, telling us that $V$ in fact admits this decomposition

In [15]:

for n in range(2,4):
    for k in range(2,4):
        print "Using n = "+str(n)+" and k = "+str(k)+": "

        Sk=SymmetricGroup(k)
        CSk=SymmetricGroup(k).algebra(QQbar)
        CSk_idem=CSk.central_orthogonal_idempotents()
        CSk_basis=[p for p in SymmetricGroup(k)]
        mat_list=[]
        for s in CSk_idem:
            mat_list.append(rho(s,n,k))
        dim_sum=0
        for M in mat_list:
            dim_sum+=M.rank()
            Mcol=M.column_space()
            for N in mat_list:
                if M!=N:
                    Ncol=N.column_space()
                    IntSpace=Mcol.intersection(Ncol)
                    if IntSpace.dimension()!=0:
                        print "ERROR"
                        break
            for g in Sk.gens():
                gMat=rho_local(g,n,k)
                Mact=(gMat*M).column_space()
                if not Mact.is_subspace(Mcol):
                    print "Error"
                    break
        if dim_sum != n^k:
            print "Error"
            break
        print "All checks out!"
        


Using n = 2 and k = 2: 
All checks out!
Using n = 2 and k = 3: 
All checks out!
Using n = 3 and k = 2: 
All checks out!
Using n = 3 and k = 3: 
All checks out!
